# Activity 5

This activity is an application of the Newton-Raphson method for univariate root-finding.

## Imports

If you wish to import other packages and/or modules,
add a blank line after importing `scipy.optimize` in the following code cell,
followed by the comment line `# Custom imports`,
after which your additional imports should be disclosed.
For example, to use NumPy, the succeeding code cell should show

>> ```python
>> import math as mt
>> 
>> import scipy.optimize as spo
>> 
>> # Custom imports
>> import numpy as np
>> ```

In [1]:
import math as mt

import scipy.optimize as spo

# Custom imports
import numpy as np

## Scenario

Suppose you have a second-order circuit that can be reduced into
an equivalent resistance $R$
parallel-connected to
an equivalent inductance $L$
as well as to
an equivalent capacitance $C$
(see [diagram](./act-05.png)).
The said model is only valid over the interval $4 \leq t \leq 21$ seconds.

Determine:
- the minimum voltage across the equivalent-inductance element,
  and
- when the voltage across the equivalent-resistance element is a minimum,

for when:
- $C = 500$ millifarads,
- $R = \frac{2}{3}$ ohms,
  and
- $L = 1000$ millihenrys,

provided that 
$V = 10$ volts
and
$I = 2$ amperes
at the start of the applicable interval.

## Modelling

Derive $f\!\left(t\right)$ whose root corresponds to when the voltage across the equivalent-resistance element is a minimum.
Then, derive an expression of $V\!\left(t\right)$ across the equivalent-inductance element.

Construct three Python functions:
- `f()`, which implements $f\!\left(t\right)$;
- `df()`, which implements the derivative of $f\!\left(t\right)$;
  and
- `v_L()`, which implements $V\!\left(t\right)$.

Each of the said functions should take a single positional argument `t`.

In the succeeding code cell,
replace the `pass` statement with your code.

Solution: 

Applying KCL on the top node:
$$i_L(t) + i_R(t) + i_C(t) = 0$$

Express each branch current in terms of \(V\):
$$i_L(t) + \frac{V(t)}{R} + C\frac{dV(t)}{dt} = 0$$

Differentiate the KCL with respect to time (t) (so that $i_L(t)$ is replaced by $\frac{di_L(t)}{dt}\ = \frac{V(t)}{L}\ $:
$$\frac{1}{L}V(t) + \frac{1}{R}\frac{dV(t)}{dt} + C\frac{d^2V(t)}{dt^2} = 0$$

Dividing both sides by C, rearranging, and substituting the given:
$$\frac{d^2V(t)}{dt^2} + \frac{1}{RC}\frac{dV(t)}{dt} + \frac{1}{LC}V(t) = 0$$
$$\frac{d^2V(t)}{dt^2} + 3\frac{dV(t)}{dt} + 2V(t) = 0$$

Since this is a simple parallel RLC circuit, we can define $a$ and $w_0$:
$$\alpha = \frac{1}{2RC} = \frac{3}{2} \, s^{-1}$$
$$\omega_0 = \frac{1}{\sqrt{LC}} = \sqrt{2} \, s^{-1}$$

Since $a$ is > $w_0$ the system is overdamped with two real and distinct roots:
$$s_{1,2} = -\alpha \pm \sqrt{\alpha^2 - \omega_0^2}$$
$$s_1 = -1 \quad s_2 = -2$$

Thus the general solution for the voltage (using time origin shifted to the start of the model interval $t_0 = 4s$; defining $τ = t-t_0$:
$$V(τ) = Ae^{-τ} + Be^{-2τ}$$
$$V(t) = Ae^{-(t-4)} + Be^{-2(t-4)}$$

Solving for constants A and B, we apply the predetermined conditions.  
Let the initial capacitor voltage at $t_0 = 4s$ be $V_0 = V(4)$, and the initial current $I_0=i_L(4)$. Then at $τ = 0$: 

$$V(0) = A + B = 10V →Eq.1$$

Differentiating $V(t)$:
$$V'(t) = -Ae^{-\tau} - 2Be^{-2\tau}$$

From capacitor current:
$$C V'(0) = i_C(0) = -(i_L(0) + i_R(0)) = -\frac{V(0)}{R}\ - I_0$$
$$V'(0) = -\frac{1}{C}\ (\frac{V(0)}{R}\ + \frac{1}{C}\ I_0)$$

Substituting:
$$V'(0) = -2\left(\frac{1}{3}(V(0) + I(0))\right) = -(3V(0) + 2I(0)) = -34V$$
$$V'(0) = -A - 2B = -34 → Eq. 2$$

Two equations, two unknowns:
$$A = -14$$
$$B = 24$$

Hence,
$$V(\tau) = -14e^{-\tau} + 24e^{-2\tau}$$
$$V(t) = -14e^{-(t-4)} + 24e^{-2(t-4)}$$

A minimum of $V(t)$ occurs when $\frac{dV(t)}{dt} = 0 $; that is our $f(t)$:
$$f(t) = \frac{dV(t)}{dt} = (-14e^{-(t-4)} + 24e^{-2(t-4)})$$
$$f'(t) = 14e^{-(t-4)} - 48e^{-2(t-4)}$$

solving for the values of $A$ and $B$\
$A= -14e^{4}$ and $B= 24e^{8}$ 

In [2]:
C = 0.5 # Farad
R = 2/3 # Ohm
L = 1   # Henry

S_1, S_2 = -1, -2 # damping factors
A, B = -14*mt.exp(4), 24*mt.exp(8)

def V_L(t):    # V_L(t) = V(t) across the circuit
    V = A*mt.exp(S_1*t) + B*mt.exp(S_2*t) 
    return V

def resid_from_x(t):   # V'(t) = f(t), where it is at minimum.
    V_prime = S_1*A*mt.exp(S_1*t) + S_2*B*mt.exp(S_2*t)
    return V_prime

def resid_from_x_prime(t):   # V"(t) = f'(t) for the newton search derivative.
    return S_1*S_1*A*mt.exp(S_1*t) + S_2*S_2*B*mt.exp(S_2*t)

Use the Newton-Raphson method to find the root of $f\!\left(t\right)$.

Define the Python variable `T0` that will store the initial estimate of the root.
Run
[`scipy.optimize.newton()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html)
such that you get the (approximate) root as well as convergence information,
and store the said outputs in Python variables `t_nr` and `t_nr_info`, respectively.
You may provide non-default values to the tolerances and to the iteration budget.

In the succeeding code cell,
replace the `pass` statement with your code.

In [3]:
T0 = 4
MAX_I = 300
tolerance = 1e-6

t_nr, t_nr_info = spo.newton(resid_from_x, T0, fprime = resid_from_x_prime, tol = tolerance, maxiter = MAX_I, full_output = True, disp = False)
print(f"Finding a root starting at {T0}:")

Finding a root starting at 4:


In [4]:
print(t_nr_info)



      converged: True
           flag: converged
 function_calls: 16
     iterations: 8
           root: 5.232143681292632
         method: newton


## Results and discussion

Verify if the obtained (approximate) root is indeed the root
by printing the corresponding value of $f\!\left(t\right)$.
The printed display should follow the following format

>> Residual value: 1.23456790e-07

where 1.23456790e-07 is just an exemplar value.

In the succeeding code cell,
replace the `pass` statement with your code.

In [5]:
print(f"Residual value: {resid_from_x(t_nr)}") 

Residual value: -1.7763568394002505e-15


Print the minimum voltage across the equivalent-inductance element,
such that the output follows the following:

>> The minimum voltage across the equivalent-inductance element is 1.234567 volts.

where 1.234567 is just an examplar value for the said minimum voltage.

In the succeeding code cell,
replace the `pass` statement with your code.

In [6]:
print(f"The minimum voltage across the equivalent-inductance element is {V_L(t_nr)} volts.")

The minimum voltage across the equivalent-inductance element is -2.0416666666666665 volts.


## Submission instructions

Download this notebook file,
and save with a filename following the pattern
`ACT-05_<section>_<ID number>`,
where the section is as reflected in your Google Classroom.
For example, if your ID number is 2013-0024
and you are enrolled to the M34W12 class,
then your notebook should be named `ACT-05_M34W12_2013-0024.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-10-09*